In [1]:
import yfinance as yf
import pandas as pd
from tr4der.tr4der import Tr4der

query = "I want to use a neural network to trade uber for the past 4 years"

inst = Tr4der(query)


filtered_stocks = self._ticker_data[(self._ticker_data['ticker'] == 'UBER')]
import yfinance as yf
import pandas as pd

tickers = ['UBER']
start_date = '2019-10-19'
end_date = '2023-10-19'
_strategy_data = pd.DataFrame()

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    _strategy_data[ticker] = data['Close']
Loading data...


[*********************100%***********************]  1 of 1 completed


{'long': <function SimpleStrategies.long at 0x12426d870>, 'long_short': <function SimpleStrategies.long_short at 0x12426d990>, 'short': <function SimpleStrategies.short at 0x12426d900>, 'macd_trend_following': <function TechnicalAnalysisStrategies.macd_trend_following at 0x1242adf30>, 'mean_reversion_bollinger_bands': <function TechnicalAnalysisStrategies.mean_reversion_bollinger_bands at 0x1242ae050>, 'mean_reversion_moving_average': <function TechnicalAnalysisStrategies.mean_reversion_moving_average at 0x1242adfc0>, 'momentum_strategy': <function TechnicalAnalysisStrategies.momentum_strategy at 0x1242adea0>, 'pairs_trading': <function TechnicalAnalysisStrategies.pairs_trading at 0x1242ae0e0>, 'LSTM': <function MachineLearningStrategies.LSTM at 0x1257b3010>, 'decision_tree_regression': <function MachineLearningStrategies.prepare_data.<locals>.wrapper at 0x1257b2dd0>, 'linear_regression': <function MachineLearningStrategies.prepare_data.<locals>.wrapper at 0x1257b2a70>, 'neural_network

RuntimeError: Error executing strategy code: name 'class_name' is not defined

In [ ]:
df = inst.strategy_data

df.dropna(inplace=True)

In [ ]:
import re
import ta
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
import numpy as np

ticker = "UBER"
technical_indicators = ['SMA_20', 'SMA_50', 'EMA_20', 'EMA_50', 'RSI_14']

# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.5, random_state=0, shuffle=False)

# Separate features and target variable
print(train_data.columns)
train_features = train_data[technical_indicators + ['Previous_Day_Return']]
train_target = train_data[f'{ticker}_return']

# Create and train the SVR model
mlp = MLPRegressor(hidden_layer_sizes=(100,100), max_iter=1000, shuffle=False)
mlp.fit(train_features, train_target)

# Make predictions on the test set
test_features = test_data[technical_indicators + ['Previous_Day_Return']]
test_target = test_data[f'{ticker}_return']
predictions = mlp.predict(test_features)
test_data['predictions'] = predictions


#We actually find a very good negative correlation between this ML model. 
#Let's add signals for buying/selling
test_data[f'{ticker}_signal'] = np.where(test_data['predictions'] > 0, 'Buy', 'Sell')

test_data[f'{ticker}_position'] = np.where(test_data[f'{ticker}_signal'] == 'Buy', 1, -1)
test_data[f'Total_Return'] = test_data[f'{ticker}_position'] * test_data[f'{ticker}_return']
test_data[f'Cumulative_Return'] = (1 + test_data[f'Total_Return']).cumprod()
test_data[f'Drawdown'] = (test_data[f'Cumulative_Return'] / test_data[f'Cumulative_Return'].cummax()) - 1

from sklearn.metrics import r2_score

r_squared = r2_score(test_target, predictions)
print(f"R-squared: {r_squared}")
    

In [ ]:
test_data.tail(50)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

# ... existing code ...

# Calculate the error for each prediction
test_data['error'] = test_data[f'{ticker}_return'] - test_data['predictions']

# Calculate Mean Squared Error
mse = mean_squared_error(test_target, predictions)
print(f"Mean Squared Error: {mse}")

# Plot the training error over iterations
plt.figure(figsize=(14, 7))
plt.plot(mlp.loss_curve_, label='Training Error')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Neural Network Training Error Over Iterations')
plt.legend()
plt.show()


# ... existing code ...

In [ ]:
def mydec(func):
    def wrapper(*args, **kwargs):
        print("Starting")
        result = func(*args, **kwargs)
        print("Ending")
        return result
    return wrapper

@mydec
